<a href="https://colab.research.google.com/github/cowiety/energy-forecast/blob/main/LSTM_dataset/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Model

import statements

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd

open data files and convert to numpy arrays

In [5]:
# open uploaded csv file
demand_file = open('Demand Data (2021-12-03).csv')
# generate numpy array from file
demand = np.genfromtxt(demand_file, delimiter=',', skip_header=1, usecols=(1))

weather_file = open('Historical Weather Data (2021-12-03).csv')
weather = np.genfromtxt(weather_file, delimiter=',', skip_header=1, usecols=(1,2,3,4))

holiday_file = open('isHoliday (2021-12-03).csv')
holiday = np.genfromtxt(holiday_file, delimiter=',', skip_header=1, usecols=(1))

split the 2D weather array into 4 x 1D arrays

In [6]:
# apparently there's an empty row at the bottom that drove me insane for 2 hours
weather = weather[:-1]
# splitting weather array into a dictionary called weather_comp
weather_comp = np.split(weather, 4, axis=1)
# putting each component into its own variable
clouds = weather_comp[0]
temp = weather_comp[1]
humidity = weather_comp[2]
wind = weather_comp[3]

combine all the features into one array

In [7]:
data = np.column_stack((demand, holiday, clouds, temp, humidity, wind))
print(data)

[[ 1.9436e+04  0.0000e+00  7.2310e+01  4.1000e-01  9.2320e+01  3.8620e+00]
 [ 1.9734e+04  0.0000e+00  7.2950e+01  1.1800e+00  9.1990e+01  4.4910e+00]
 [ 1.9994e+04  0.0000e+00  9.5480e+01  1.7600e+00  9.0820e+01  3.7060e+00]
 ...
 [ 1.9936e+04  0.0000e+00  7.4830e+01  3.6000e-01  5.9410e+01  2.2120e+00]
 [ 1.9384e+04  0.0000e+00  7.7760e+01  5.0000e-02  6.1090e+01  2.2120e+00]
 [ 1.8206e+04  0.0000e+00  7.7760e+01 -4.4000e-01  6.3850e+01  2.2120e+00]]


reshape into a 3D array to prepare the data for LSTM model

In [8]:
y = data[:, 0]
X = data[:, 1:5]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
print(X_train.shape)

(16633, 4)


In [17]:
X_train = X_train.reshape(1, 16633, 4)
y_train = y_train.reshape(1, 16633, 1)
X_test = X_test.reshape(1, 8193, 4)
y_test = y_test.reshape(1, 8193, 1)
print(X_test.shape)

(1, 8193, 4)


creating a RNN with one hidden LSTM layer and one Dense output layer

In [21]:
model = tf.keras.Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]), activation = 'relu', return_sequences=True))
model.add(Dense(1, activation='softmax'))
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test), verbose=2, shuffle=False)

Epoch 1/3


ValueError: ignored